In [1]:
# Sel 1: Import Library Utama dan Setup Path
import os
import sys
import random
import numpy as np
import pandas as pd
import cv2 # Untuk operasi gambar/video jika diperlukan di sini
import tensorflow as tf
from ultralytics import YOLO # Untuk YOLO
from collections import deque # Untuk sekuens fitur

# Setup Path (sesuaikan jika notebook tidak di root proyek)
# Jika notebook ada di 'notebooks/' dan 'src/' ada di level yang sama dengan 'notebooks/'
# Maka '../' akan membawa kita ke root proyek, lalu kita bisa masuk ke 'src'
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) 
# Jika notebook dijalankan dari root proyek (misal, smart-cctv/), maka:
# project_root = os.getcwd() 

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

data_path = os.path.join(project_root, "data") # Path ke direktori data utama

print(f"Project Root: {project_root}")
print(f"Source Path (untuk impor): {src_path}")
print(f"Data Path: {data_path}")

# Nonaktifkan pesan log TensorFlow yang terlalu verbose (opsional)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
tf.get_logger().setLevel('ERROR')

# Reproducibility (opsional, tapi baik untuk eksperimen)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("Library utama dan path berhasil di-setup.")

Project Root: d:\Earth\Smart-CCTV
Source Path (untuk impor): d:\Earth\Smart-CCTV\src
Data Path: d:\Earth\Smart-CCTV\data
Library utama dan path berhasil di-setup.


In [2]:
# Sel 2: Import Modul-Modul Proyek
# (Pastikan file __init__.py ada di direktori src/ dan src/data/ serta src/models/ agar bisa diimpor sebagai package)

# --- Dari src.data ---
#
try:
    from data.load_data import process_loaded_data
    from data.clean_data import clean_missing_values, remove_duplicates, check_file_paths_exist 
    from data.split_data import split_data_into_sets
    print("Modul data berhasil diimpor.")
except ImportError as e:
    print(f"Error mengimpor modul data: {e}. Pastikan sys.path sudah benar dan ada __init__.py.")

# --- Dari src.models (YOLO-CNN-LSTM) ---
# Kita asumsikan Anda menyimpan YoloCnnLstmProcessor di file yolo_cnn_lstm_model.py
# dan fungsi build_and_train_model_for_ga di model_builder_ga.py
try:
    from models.yolo.yolo_cnn_lstm_model import YoloCnnLstmProcessor  # Atau apapun nama kelas/file yang Anda gunakan
    from models.genetika import build_and_train_model_for_ga # Fungsi ini juga akan memanggil model dari yolo_cnn_lstm_model.py
    print("Modul model (YOLO-CNN-LSTM & builder GA) berhasil diimpor.")
except ImportError as e:
    print(f"Error mengimpor modul model: {e}. Pastikan nama file dan kelas sudah benar, serta sys.path.")


# --- Dari src.models.genetika ---
#
try:
    # Impor fungsi utama GA dan fungsi pendukungnya
    from models.genetika.genetika import (
        HYPERPARAMETER_RANGES, # Penting! Ini harus didefinisikan di genetika.py
        encode_value,
        decode_binary_segment,
        generate_random_hyperparameters,
        encode_hyperparameters_to_chromosome,
        decode_chromosome_to_hyperparameters,
        pembentukan_populasi_awal,
        evaluasi_fitness, # Ini akan memanggil build_and_train_model_for_ga
        roulette_wheel_selection,
        crossover_operator,
        mutation_operator,
        algoritma_genetika_yolo_cnn_lstm # Fungsi utama GA
    )
    print("Modul genetika berhasil diimpor.")
except ImportError as e:
    print(f"Error mengimpor modul genetika: {e}. Pastikan semua fungsi didefinisikan dan HYPERPARAMETER_RANGES ada.")
except NameError as ne:
    print(f"NameError saat impor modul genetika (kemungkinan HYPERPARAMETER_RANGES belum didefinisikan di genetika.py): {ne}")

Modul data berhasil diimpor.
Error mengimpor modul model: cannot import name 'Input' from 'tensorflow' (d:\Earth\Smart-CCTV\venv\Lib\site-packages\tensorflow\__init__.py). Pastikan nama file dan kelas sudah benar, serta sys.path.
Error mengimpor modul genetika: cannot import name 'Model' from 'tensorflow' (d:\Earth\Smart-CCTV\venv\Lib\site-packages\tensorflow\__init__.py). Pastikan semua fungsi didefinisikan dan HYPERPARAMETER_RANGES ada.


In [ ]:
# Sel 3: Definisi Konstanta dan Konfigurasi Eksperimen

# --- Konfigurasi Data ---
# Ganti dengan path ke data Anda atau gunakan path untuk data dummy
RAW_DATA_DIR = os.path.join(data_path, "raw_dummy") # Contoh: data/raw_dummy
ANNOTATIONS_FILE = os.path.join(RAW_DATA_DIR, "annotations_dummy.csv") # Contoh
PROCESSED_DATA_DIR = os.path.join(data_path, "processed_dummy")
FINAL_DATA_DIR = os.path.join(data_path, "final_dummy")

# Pastikan direktori dummy ada untuk menghindari error
os.makedirs(RAW_DATA_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
os.makedirs(FINAL_DATA_DIR, exist_ok=True)

# --- Konfigurasi Model (Contoh untuk pemanggilan langsung jika tidak lewat GA) ---
YOLO_MODEL_NAME = "yolov8n.pt"
CNN_INPUT_SIZE_HW_MODEL = (64, 64) # Ukuran input untuk CNN per objek
CNN_FEATURE_DIM_MODEL = 128        # Dimensi output fitur dari CNN
LSTM_SEQ_LEN_MODEL = 10            # Panjang sekuens untuk input LSTM
LSTM_UNITS_MODEL = 32
NUM_CLASSES_BEHAVIOR = 3           # Contoh: normal, aneh, berbahaya
INPUT_SHAPE_PER_FRAME_MODEL = (CNN_INPUT_SIZE_HW_MODEL[0], CNN_INPUT_SIZE_HW_MODEL[1], 3) # h, w, c

# --- Konfigurasi Algoritma Genetika ---
# Ini akan digunakan oleh fungsi algoritma_genetika_yolo_cnn_lstm
GA_JUMLAH_KROMOSOM = 6 # Ukuran populasi (kecil untuk eksperimen cepat)
GA_GENERATIONS = 3     # Jumlah generasi (kecil untuk eksperimen cepat)
GA_CROSSOVER_RATE = 0.7
GA_MUTATION_RATE = 0.1
GA_EPOCHS_PER_EVAL = 3 # Jumlah epoch untuk melatih setiap model kandidat di GA (sangat kecil)
GA_ELITE_SIZE = 1

print("Konfigurasi eksperimen disiapkan.")
# Periksa apakah HYPERPARAMETER_RANGES sudah terdefinisi (diimpor dari genetika.py)
try:
    if HYPERPARAMETER_RANGES:
        print(f"HYPERPARAMETER_RANGES ditemukan dengan {len(HYPERPARAMETER_RANGES)} parameter.")
except NameError:
    print("PENTING: HYPERPARAMETER_RANGES belum didefinisikan/diimpor! GA tidak akan berjalan dengan benar.")
    HYPERPARAMETER_RANGES = {} # Definisikan dummy jika perlu agar notebook tidak error

In [ ]:
# Sel 4: Persiapan Data (Menggunakan Fungsi dari src.data)

print("--- TAHAP 1: PEMUATAN DATA ---")
# Membuat data dummy jika file anotasi tidak ada
if not os.path.exists(ANNOTATIONS_FILE):
    print(f"File anotasi dummy tidak ditemukan di {ANNOTATIONS_FILE}. Membuat file dummy...")
    dummy_video_files = []
    for i in range(20): # Buat 20 entri data dummy
        vid_name = f"dummy_vid_{i+1}.mp4"
        # Buat file video dummy kosong jika ingin check_file_paths_exist berfungsi
        # open(os.path.join(RAW_DATA_DIR, vid_name), 'a').close() 
        dummy_video_files.append(vid_name)

    dummy_df_content = {
        'filename': dummy_video_files,
        'label_perilaku': [random.choice(['normal', 'aneh', 'berbahaya']) for _ in range(20)],
        'durasi_detik': [random.randint(5, LSTM_SEQ_LEN_MODEL + 10) for _ in range(20)] # Pastikan durasi cukup
    }
    pd.DataFrame(dummy_df_content).to_csv(ANNOTATIONS_FILE, index=False)
    print(f"File anotasi dummy dibuat dengan {len(dummy_video_files)} entri.")

# Memuat data (sekarang menggunakan file dummy yang mungkin baru dibuat)
loaded_dataframe = process_loaded_data(raw_data_path=RAW_DATA_DIR, 
                                       annotation_file_path=ANNOTATIONS_FILE)

if not loaded_dataframe.empty:
    print(f"\nData berhasil dimuat (atau dummy) ({len(loaded_dataframe)} baris):")
    print(loaded_dataframe.head())
    
    # Simpan output pemuatan data (opsional untuk notebook, tapi baik untuk alur kerja)
    loaded_dataframe.to_csv(os.path.join(PROCESSED_DATA_DIR, "01_loaded_data.csv"), index=False)
else:
    print("Pemuatan data gagal atau menghasilkan DataFrame kosong.")

print("-" * 50)

print("\n--- TAHAP 2: PEMBERSIHAN DATA ---")
if not loaded_dataframe.empty:
    # Untuk demo, kita asumsikan video path tidak perlu dicek karena dummy
    # Jika data asli, aktifkan:
    # cleaned_df_step1 = check_file_paths_exist(loaded_dataframe, path_column='video_path') 
    cleaned_df_step1 = loaded_dataframe.copy() 

    if 'label_perilaku' in cleaned_df_step1.columns:
        cleaned_df_step2 = clean_missing_values(cleaned_df_step1, columns_to_check=['video_path', 'label_perilaku'])
    else:
        cleaned_df_step2 = clean_missing_values(cleaned_df_step1, columns_to_check=['video_path'])
        print("Peringatan: Kolom 'label_perilaku' tidak ada untuk pemeriksaan missing values.")


    cleaned_dataframe = remove_duplicates(cleaned_df_step2, subset_cols=['filename'])
    
    print(f"\nData setelah dibersihkan ({len(cleaned_dataframe)} baris):")
    print(cleaned_dataframe.head())
    cleaned_dataframe.to_csv(os.path.join(PROCESSED_DATA_DIR, "02_cleaned_data.csv"), index=False)
else:
    print("Tidak ada data yang dimuat untuk dibersihkan.")
    cleaned_dataframe = pd.DataFrame()
print("-" * 50)


print("\n--- TAHAP 3: PEMBAGIAN DATA ---")
if not cleaned_dataframe.empty:
    stratify_col_name = 'label_perilaku' if 'label_perilaku' in cleaned_dataframe.columns else None
    train_df, val_df, test_df = split_data_into_sets(
        cleaned_dataframe, 
        test_size=0.2, 
        val_size=0.15, 
        random_state=SEED, 
        stratify_col=stratify_col_name
    )
    print(f"\nUkuran set: Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")
    
    # Simpan set data
    if not train_df.empty: train_df.to_csv(os.path.join(FINAL_DATA_DIR, "train_set.csv"), index=False)
    if not val_df.empty: val_df.to_csv(os.path.join(FINAL_DATA_DIR, "validation_set.csv"), index=False)
    if not test_df.empty: test_df.to_csv(os.path.join(FINAL_DATA_DIR, "test_set.csv"), index=False)
    print("Set data telah disimpan.")
else:
    print("Tidak ada data bersih untuk dibagi.")
    train_df, val_df, test_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
print("-" * 50)

In [ ]:
# Sel 5: Persiapan Data Input untuk Model (Placeholder/Dummy)
# Di dunia nyata, ini akan melibatkan ekstraksi frame dari video, cropping objek, dll.
# Untuk eksperimen GA, kita butuh data (X_train_seq, y_train_seq, X_val_seq, y_val_seq)

def create_dummy_sequence_data(num_samples, seq_length, feature_dim, num_classes):
    """Membuat data sekuens dummy untuk pelatihan/validasi LSTM."""
    X = np.random.rand(num_samples, seq_length, feature_dim).astype('float32')
    # Untuk klasifikasi, y bisa berupa integer atau one-hot
    y_labels = np.random.randint(0, num_classes, num_samples)
    if num_classes > 1 :
        y = tf.keras.utils.to_categorical(y_labels, num_classes=num_classes)
    else: # Binary
        y = y_labels.astype('float32')
    return X, y

print("--- Menyiapkan Data Sekuens Dummy untuk Model GA ---")
# Asumsikan kita sudah punya data tabular (train_df, val_df)
# Kita perlu mengubahnya menjadi data sekuensial untuk input ke CNN-LSTM
# Ini adalah bagian yang paling kompleks dan spesifik untuk data Anda.
# Fitur dari CNN akan menjadi input sekuens untuk LSTM.

# Untuk tujuan demonstrasi GA, kita akan membuat data sekuens numerik dummy.
# Dimensi fitur akan sesuai dengan output CNN (CNN_FEATURE_DIM_MODEL).
# Panjang sekuens akan diambil dari LSTM_SEQ_LEN_MODEL (ini bisa dioptimasi GA juga).

# Jumlah sampel dummy (sesuaikan dengan ukuran train_df dan val_df jika memungkinkan)
num_train_samples = len(train_df) if not train_df.empty else 20 # Min 20 sampel dummy
num_val_samples = len(val_df) if not val_df.empty else 5     # Min 5 sampel dummy

if num_train_samples > 0 and num_val_samples > 0:
    # Input untuk fungsi build_and_train_model_for_ga adalah fitur sekuensial
    # hasil dari CNN. Jadi, feature_dim di sini adalah output dari CNN.
    # Bentuk X: (num_samples, seq_length, cnn_output_height, cnn_output_width, cnn_output_channels)
    # Atau jika fitur sudah diekstrak: (num_samples, seq_length, cnn_feature_dim)
    
    # Untuk build_and_train_model_for_ga yang kita rancang, ia menerima:
    # train_X_seq, train_y_seq, val_X_seq, val_y_seq
    # di mana X_seq adalah (num_samples, seq_length, height, width, channels)
    # Ini berarti fungsi tersebut akan menangani CNN di dalamnya.

    # Jadi, kita buat data dummy dengan shape (num_samples, LSTM_SEQ_LEN_MODEL, CNN_INPUT_SIZE_HW_MODEL[0], CNN_INPUT_SIZE_HW_MODEL[1], 3)
    
    print(f"Membuat data latih dummy: {num_train_samples} sampel, seq_len={LSTM_SEQ_LEN_MODEL}, shape_per_frame={INPUT_SHAPE_PER_FRAME_MODEL}")
    X_train_seq_dummy, y_train_seq_dummy = create_dummy_sequence_data(
        num_train_samples, LSTM_SEQ_LEN_MODEL, 
        INPUT_SHAPE_PER_FRAME_MODEL[0] * INPUT_SHAPE_PER_FRAME_MODEL[1] * INPUT_SHAPE_PER_FRAME_MODEL[2], # Flattened features for simplicity
        NUM_CLASSES_BEHAVIOR
    )
    # Reshape X_train_seq_dummy ke (num_samples, seq_length, height, width, channels)
    X_train_seq_dummy = X_train_seq_dummy.reshape(
        num_train_samples, LSTM_SEQ_LEN_MODEL, 
        INPUT_SHAPE_PER_FRAME_MODEL[0], INPUT_SHAPE_PER_FRAME_MODEL[1], INPUT_SHAPE_PER_FRAME_MODEL[2]
    )


    print(f"Membuat data validasi dummy: {num_val_samples} sampel.")
    X_val_seq_dummy, y_val_seq_dummy = create_dummy_sequence_data(
        num_val_samples, LSTM_SEQ_LEN_MODEL,
        INPUT_SHAPE_PER_FRAME_MODEL[0] * INPUT_SHAPE_PER_FRAME_MODEL[1] * INPUT_SHAPE_PER_FRAME_MODEL[2], # Flattened
        NUM_CLASSES_BEHAVIOR
    )
    X_val_seq_dummy = X_val_seq_dummy.reshape(
        num_val_samples, LSTM_SEQ_LEN_MODEL,
        INPUT_SHAPE_PER_FRAME_MODEL[0], INPUT_SHAPE_PER_FRAME_MODEL[1], INPUT_SHAPE_PER_FRAME_MODEL[2]
    )

    print(f"Shape X_train_seq_dummy: {X_train_seq_dummy.shape}, y_train_seq_dummy: {y_train_seq_dummy.shape}")
    print(f"Shape X_val_seq_dummy: {X_val_seq_dummy.shape}, y_val_seq_dummy: {y_val_seq_dummy.shape}")
else:
    print("Tidak cukup data dari train_df/val_df untuk membuat sampel dummy. Lewati pembuatan data sekuens.")
    X_train_seq_dummy, y_train_seq_dummy, X_val_seq_dummy, y_val_seq_dummy = None, None, None, None

print("-" * 50)

In [ ]:
# Sel 6: Pengujian Algoritma Genetika (dengan build_and_train_model_for_ga placeholder)

print("--- TAHAP 4: EKSPERIMEN ALGORITMA GENETIKA ---")

# Pastikan HYPERPARAMETER_RANGES sudah terdefinisi dan diimpor
# Pastikan model_builder_ga.py berisi fungsi build_and_train_model_for_ga
# dan genetika.py berisi algoritma_genetika_yolo_cnn_lstm serta fungsi pendukung

if X_train_seq_dummy is not None and y_train_seq_dummy is not None and \
   X_val_seq_dummy is not None and y_val_seq_dummy is not None and \
   HYPERPARAMETER_RANGES: # Pastikan data dan ranges ada

    print("Memulai Algoritma Genetika (ini bisa memakan waktu, bahkan dengan data dummy dan epoch sedikit)...")
    
    # Sebelum memanggil algoritma_genetika_yolo_cnn_lstm, pastikan fungsi evaluasi_fitness
    # di dalam genetika.py sudah benar memanggil build_and_train_model_for_ga
    # dan build_and_train_model_for_ga bisa menangani hyperparameter yang diberikan.
    
    # Kita perlu memastikan 'build_and_train_model_for_ga' diimpor dan digunakan dengan benar
    # oleh 'evaluasi_fitness' di dalam 'genetika.py'.
    # Untuk eksperimen ini, kita akan langsung memanggil algoritma_genetika_yolo_cnn_lstm
    # yang seharusnya sudah menggunakan semua komponen tersebut.

    # Definisikan argumen untuk build_and_train_model_for_ga (yang akan dipass oleh GA)
    # seperti input_shape_per_frame dan num_classes
    # Ini akan menjadi argumen tambahan yang di-pass ke evaluasi_fitness jika perlu.
    # Namun, fungsi GA kita sudah menerimanya.
    
    best_individual_ga, fitness_history_ga = algoritma_genetika_yolo_cnn_lstm(
        train_X_seq_dummy, y_train_seq_dummy,
        X_val_seq_dummy, y_val_seq_dummy,
        input_shape_per_frame=INPUT_SHAPE_PER_FRAME_MODEL, # Ditambahkan untuk build_and_train_model_for_ga
        num_classes=NUM_CLASSES_BEHAVIOR,                  # Ditambahkan
        jumlah_kromosom=GA_JUMLAH_KROMOSOM,
        generations=GA_GENERATIONS,
        crossover_rate=GA_CROSSOVER_RATE,
        mutation_rate=GA_MUTATION_RATE,
        epochs_per_eval=GA_EPOCHS_PER_EVAL,
        elite_size=GA_ELITE_SIZE
    )

    if best_individual_ga:
        print("\n--- Hasil Optimasi Algoritma Genetika ---")
        print(f"Fitness Terbaik (Val Accuracy): {best_individual_ga['fitness']:.4f}")
        print("Hyperparameter Terbaik:")
        for key, value in best_individual_ga['hyperparameters'].items():
            print(f"  {key}: {value}")
        # print(f"Chromosome Terbaik: {best_individual_ga['chromosome']}") # Bisa sangat panjang
        
        # Plotting fitness history (opsional)
        import matplotlib.pyplot as plt
        if fitness_history_ga:
            plt.figure(figsize=(10, 6))
            plt.plot(range(1, len(fitness_history_ga) + 1), fitness_history_ga, marker='o')
            plt.title('Riwayat Fitness Terbaik per Generasi')
            plt.xlabel('Generasi')
            plt.ylabel('Fitness (Validation Accuracy)')
            plt.grid(True)
            plt.show()
    else:
        print("Optimasi GA tidak menghasilkan individu terbaik (mungkin terjadi error atau data tidak cukup).")
else:
    print("Data sekuens dummy atau HYPERPARAMETER_RANGES tidak tersedia. Lewati eksperimen GA.")

print("-" * 50)

In [ ]:
# Sel 7: Pengujian YoloCnnLstmProcessor (Pemrosesan Sekuens Video - Placeholder)
# Ini adalah pengujian terpisah dari GA, untuk melihat apakah alur pemrosesan video berjalan.
# Akan menggunakan bobot acak jika tidak ada path bobot yang diberikan.

print("--- TAHAP 5: EKSPERIMEN PEMROSESAN VIDEO DENGAN YOLO-CNN-LSTM (Placeholder) ---")

# Konfigurasi untuk YoloCnnLstmProcessor
YOLO_MODEL_PROC = "yolov8n.pt"
CNN_WEIGHTS_PROC = None # Path jika ada
LSTM_WEIGHTS_PROC = None # Path jika ada
CNN_INPUT_SIZE_PROC = (64, 64)
CNN_FEATURE_DIM_PROC = 128 # Harus sama dengan yang diharapkan LSTM
LSTM_SEQ_LEN_PROC = 10     # Harus sama dengan input shape LSTM
LSTM_UNITS_PROC = 32
NUM_CLASSES_PROC = NUM_CLASSES_BEHAVIOR # Sama dengan kelas perilaku
TARGET_YOLO_CLASSES_PROC = ['person', 'car'] # Fokus pada kelas ini saja
YOLO_CONF_PROC = 0.3

# Cek apakah ObjectTracker bisa diimpor dan digunakan
try:
    if ObjectTracker is None: # dari yolo_cnn_lstm_model.py
        raise ImportError("ObjectTracker tidak tersedia.")
        
    processor_instance = YoloCnnLstmProcessor(
        yolo_model_path=YOLO_MODEL_PROC,
        cnn_input_size=CNN_INPUT_SIZE_PROC,
        cnn_feature_vector_size=CNN_FEATURE_DIM_PROC,
        cnn_weights_path=CNN_WEIGHTS_PROC,
        lstm_sequence_length=LSTM_SEQ_LEN_PROC,
        lstm_units=LSTM_UNITS_PROC,
        lstm_num_classes=NUM_CLASSES_PROC,
        lstm_use_bidirectional=False,
        lstm_weights_path=LSTM_WEIGHTS_PROC,
        target_yolo_classes=TARGET_YOLO_CLASSES_PROC,
        yolo_confidence_threshold=YOLO_CONF_PROC
    )
    print("YoloCnnLstmProcessor berhasil diinisialisasi.")

    # Buat generator frame dummy untuk diuji
    def dummy_frames_for_processor(num_frames=LSTM_SEQ_LEN_PROC + 5, H=240, W=320):
        for i in range(num_frames):
            frame = np.random.randint(0, 256, (H, W, 3), dtype=np.uint8)
            # Tambahkan objek dummy yang bergerak sederhana
            obj_x = int(10 + (W - 50) * (i / num_frames))
            cv2.rectangle(frame, (obj_x, H//2 - 10), (obj_x + 20, H//2 + 10), (0,255,0), -1)
            yield frame
            
    dummy_video_gen = dummy_frames_for_processor()
    
    print("Memulai pemrosesan sekuens video dummy dengan YoloCnnLstmProcessor...")
    # Perlu diingat, bagian pencocokan fitur dengan ID objek di YoloCnnLstmProcessor
    # adalah placeholder dan perlu perbaikan untuk hasil yang akurat.
    # Untuk demo ini, kita hanya melihat apakah alurnya berjalan.
    
    results_processor = processor_instance.process_video_sequence(dummy_video_gen)
    
    print("\nHasil dari YoloCnnLstmProcessor:")
    if results_processor:
        for obj_id, preds in results_processor.items():
            print(f"  Objek ID {obj_id}: {len(preds)} prediksi LSTM")
            for p_info in preds[:2]: # Tampilkan 2 prediksi pertama
                print(f"    Frame {p_info['frame']} -> Kelas {p_info['predicted_class']}")
    else:
        print("Tidak ada output dari prosesor video.")

except ImportError as e:
    print(f"Tidak bisa menjalankan pengujian YoloCnnLstmProcessor karena error impor: {e}")
except Exception as ex:
    print(f"Terjadi error saat pengujian YoloCnnLstmProcessor: {ex}")


print("-" * 50)
print("Eksperimen Selesai.")